<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_8_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless tqdm

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
from pathlib import Path

In [ ]:
def crop_to_content(img):
    """Обрезает пустые поля вокруг символа (по белому фону)."""
    # Если изображение цветное, переводим в градации серого
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    # Бинаризация: всё светлее 250 становится белым (255), остальное — чёрным (0)
    _, thresh = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)
    # Находим координаты всех НЕбелых пикселей (255 - thresh инвертирует изображение)
    coords = cv2.findNonZero(255 - thresh)
    if coords is None:
        # Если не найдено ни одного непустого пикселя — возвращаем исходное изображение
        return img
    # Получаем прямоугольник, минимально охватывающий все найденные пиксели
    x, y, w, h = cv2.boundingRect(coords)
    if w == 0 or h == 0:
        # Если обрезка дала нулевой размер — возвращаем исходное изображение
        return img
    # Вырезаем найденный прямоугольник из исходного изображения
    cropped = img[y:y+h, x:x+w]
    return cropped

def resize_and_pad(img, size):
    """Масштабирует изображение с сохранением пропорций и центрирует на квадратном холсте."""
    h, w = img.shape[:2]
    if h == 0 or w == 0:
        # Если изображение пустое — возвращаем белое изображение нужного размера
        if len(img.shape) == 2:
            return np.full((size, size), 255, dtype=np.uint8)
        else:
            return np.full((size, size, 3), 255, dtype=np.uint8)
    # Вычисляем масштаб так, чтобы большая сторона стала равна size
    scale = size / max(h, w)
    if scale <= 0:
        # Защита от деления на ноль или отрицательного масштаба
        scale = 1.0
    new_w, new_h = int(w * scale), int(h * scale)
    if new_w == 0 or new_h == 0:
        # Если после масштабирования размер стал нулевым — возвращаем белое изображение
        if len(img.shape) == 2:
            return np.full((size, size), 255, dtype=np.uint8)
        else:
            return np.full((size, size, 3), 255, dtype=np.uint8)
    # Масштабируем изображение
    img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_NEAREST)
    # Вычисляем отступы для центрирования изображения на квадратном фоне
    pad_top = (size - new_h) // 2
    pad_bottom = size - new_h - pad_top
    pad_left = (size - new_w) // 2
    pad_right = size - new_w - pad_left
    # Цвет фона — белый (255)
    if len(img.shape) == 2:
        color = [255]
    else:
        color = [255, 255, 255]
    # Добавляем отступы (паддинг) вокруг изображения
    img_padded = cv2.copyMakeBorder(
        img_resized, pad_top, pad_bottom, pad_left, pad_right,
        cv2.BORDER_CONSTANT, value=color
    )
    return img_padded

def to_binary(img):
    """
    Преобразует изображение к чёрно-белому (только #FFFFFF и #000000),
    учитывая возможный цветной фон и цвет символа.
    """

    # Конвертируем в grayscale, если изображение цветное
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    # Оцениваем цвет фона: среднее значение в 4-х углах (по 10x10 пикселей)
    h, w = gray.shape
    corner_size = 10
    corners = [
        gray[0:corner_size, 0:corner_size],
        gray[0:corner_size, w-corner_size:w],
        gray[h-corner_size:h, 0:corner_size],
        gray[h-corner_size:h, w-corner_size:w]
    ]
    bg_color = np.mean([np.mean(c) for c in corners])

    # Если фон тёмный (меньше 127), инвертируем изображение, чтобы фон стал светлым
    if bg_color < 127:
        gray = 255 - gray

    # Применяем адаптивный порог (лучше работает при неравномерном освещении)
    binary = cv2.adaptiveThreshold(
        gray,
        maxValue=255,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY,
        blockSize=15,
        C=10
    )

    # После adaptiveThreshold символы — белые, фон — чёрный, инвертируем чтобы символы были чёрными
    binary = 255 - binary

    # Проверяем, что фон действительно белый, а символы — чёрные
    corners_bin = [
        binary[0:corner_size, 0:corner_size],
        binary[0:corner_size, w-corner_size:w],
        binary[h-corner_size:h, 0:corner_size],
        binary[h-corner_size:h, w-corner_size:w]
    ]
    bg_color_bin = np.mean([np.mean(c) for c in corners_bin])
    if bg_color_bin < 127:
        # Если фон всё ещё тёмный — инвертируем ещё раз
        binary = 255 - binary

    return binary

In [ ]:
def process_image(args):
    # Распаковка аргументов: путь к исходному изображению, путь для сохранения, размер
    src_path, dst_path, size = args
    try:
        # Читаем изображение в цвете
        img = cv2.imread(str(src_path), cv2.IMREAD_COLOR)
        if img is None:
            print(f"Invalid image: {src_path}")
            return
        # Обрезаем пустые поля
        img = crop_to_content(img)
        # Масштабируем и центрируем на квадратном фоне
        img = resize_and_pad(img, size)
        # Переводим в чёрно-белое
        img = to_binary(img)
        # Сохраняем результат
        cv2.imwrite(str(dst_path), img)
    except Exception as e:
        print(f"Error processing {src_path}: {e}")

In [ ]:
def prepare_filelist(input_dir, output_dir, size):
    tasks = []
    counters = {}  # Счётчик для каждой буквы

    # Рекурсивно обходим все файлы в input_dir
    for root, _, files in os.walk(input_dir):
        for file in files:
            # Пропускаем не-изображения
            if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                continue
            rel_path = os.path.relpath(root, input_dir)
            parts = Path(rel_path).parts
            # Структура: train/GUID/LETTER
            if len(parts) < 2:
                continue
            label = parts[-1]  # Название буквы (метка)
            dst_folder = os.path.join(output_dir, label)
            os.makedirs(dst_folder, exist_ok=True)
            # Счётчик для уникальных имён файлов
            if label not in counters:
                counters[label] = 0
            dst_name = f"{counters[label]:04d}.png"
            counters[label] += 1

            src_path = os.path.join(root, file)
            dst_path = os.path.join(dst_folder, dst_name)
            tasks.append((src_path, dst_path, size))
    return tasks

In [ ]:
def main(input_dir, output_dir, size):
    tasks = prepare_filelist(input_dir, output_dir, size)
    print(f"Всего изображений для обработки: {len(tasks)}")
    with Pool(processes=cpu_count()) as pool:
        list(tqdm(pool.imap_unordered(process_image, tasks), total=len(tasks)))
    print("Обработка завершена.")

In [ ]:
!apt-get install p7zip-full
!p7zip -d dataset_2025.7z

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 8622323 bytes (8421 KiB)

Extracting archive: dataset_2025.7z
--
Path = dataset_2025.7z
Type = 7z
Physical Size = 8622323
Headers Size = 60271
Method = LZMA:23
Solid = +
Blocks = 1

  0%     12% 572 - dataset_2025/test/698f53ee-1f45-44ac-a538-4e86fa976935/Y/3.bmp                                                                          35% 1340 - d

In [ ]:
# Параметры
INPUT_DIR = 'dataset_2025/test'
OUTPUT_DIR = 'processed_dataset/test'
SIZE = 64  # Задайте нужный размер NxN

main(INPUT_DIR, OUTPUT_DIR, SIZE)

# Параметры
INPUT_DIR = 'dataset_2025/train'
OUTPUT_DIR = 'processed_dataset/train'

main(INPUT_DIR, OUTPUT_DIR, SIZE)


Всего изображений для обработки: 1137


100%|██████████| 1137/1137 [00:01<00:00, 603.45it/s]


Обработка завершена.
Всего изображений для обработки: 2360


100%|██████████| 2360/2360 [00:02<00:00, 1037.95it/s]

Обработка завершена.


In [ ]:
!zip -r processed_dataset.zip processed_dataset/

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: processed_dataset/train/X/0017.png (deflated 4%)
  adding: processed_dataset/train/X/0057.png (deflated 3%)
  adding: processed_dataset/train/X/0001.png (deflated 1%)
  adding: processed_dataset/train/X/0051.png (deflated 2%)
  adding: processed_dataset/train/X/0003.png (stored 0%)
  adding: processed_dataset/train/X/0004.png (deflated 0%)
  adding: processed_dataset/train/X/0049.png (deflated 23%)
  adding: processed_dataset/train/X/0010.png (deflated 3%)
  adding: processed_dataset/train/X/0007.png (deflated 4%)
  adding: processed_dataset/train/X/0023.png (deflated 2%)
  adding: processed_dataset/train/X/0018.png (deflated 3%)
  adding: processed_dataset/train/X/0009.png (stored 0%)
  adding: processed_dataset/train/X/0013.png (deflated 3%)
  adding: processed_dataset/train/X/0058.png (deflated 2%)
  adding: processed_dataset/train/X/0016.png (deflated 7%)
  adding: processed_dataset/train/X/0047.png (defl